In [1]:
!pip install --upgrade tdmclient

In [ ]:
import math
import asyncio
from tdmclient import ClientAsync

In [4]:
from tdmclient import ClientAsync

client = ClientAsync()
node = await client.wait_for_node()
await node.lock()


Node 4831bb7f-ee1c-4555-89c5-7d093d9f4fa5

In [10]:
# Simulating a trajectory 
async def set_motors(node, left_speed, right_speed, duration):
    await node.set_variables({
        "motor.left.target": [left_speed],  
        "motor.right.target": [right_speed] 
    })
    await client.sleep(duration)  
    #Using client.sleep ensures the connection between your Python script and Thymio remains 
    #open and responsive while the script is paused.
    await stop_motors(node)  

# Function to stop the motors
async def stop_motors(node):
    await node.set_variables({
        "motor.left.target": [0],  
        "motor.right.target": [0]  
    })

In [15]:
await set_motors(node, 300, 300, 4) 
await set_motors(node, 300, -300, 1.5)
await set_motors(node, 300, 300, 4) 
await set_motors(node, 300, -300, 1.5)  

## Simplest Approach: 
- Robot just goes in opposite direction of the obstacle
- This is just to get used to the thymio nothing crazy or precise! 

In [94]:
OBSTACLE_THRESH_LOW = 10    # if plus petit, no significant obstacle
OBSTACLE_THRESH_HIGH = 20   # plus grand obstacle is very close

In [ ]:
""" def on_variables_changed(node, variables):
    try:
        prox = variables["prox.horizontal"]
        prox_front = prox[2]
        speed = -prox_front // 10
        #node.client.loop.create_task(set_motors(node, speed,speed)) #Possible way to do it with an asynchronous function askip 

        #await set_motors(node ,speed, speed) 
        #not possible to do this because "awaits" can only be used in asynchronous function
        #here we are in a callback function executed ina synchronous context

        node.send_set_variables(motors(speed, speed))
        #Instead of calling the async method set_variables which expects a result code in a message from the TDM
        #it just sends a message to change variables with send_set_variables without expecting any reply.
    except KeyError:
        pass  # prox.horizontal not found """

In [109]:
from tdmclient import ClientAsync

# Helper function to set motors
async def set_motors(node, left_speed, right_speed):
    await node.set_variables({
        "motor.left.target": [left_speed],
        "motor.right.target": [right_speed],
    })

# Circular motion function
async def circular_motion(node):
    print("circular motion")
    await set_motors(node, 300, 300)  # Forward
    await client.sleep(4)  # Move forward for 4 seconds
    await set_motors(node, 300, -300)  # Turn
    await client.sleep(1.5)
    await set_motors(node, 300, 300)  # Forward
    await client.sleep(4)  # Move forward for 4 seconds
    await set_motors(node, 300, -300)  # Turn
    await client.sleep(1.5)

# Function to stop motors
async def stop_motors(node):
    await set_motors(node, 0, 0)

def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }


def on_variables_changed(node, variables):
    try:
        prox = variables["prox.horizontal"]
        prox_front = prox[2]
        prox_left = prox[0]
        prox_left_front = prox[1]
        prox_right_front = prox[3]
        prox_right = prox[4]
        prox_behind_right = prox[5]
        prox_behind_left = prox[6]

        # Check for obstacles based on thresholds
        #ATTENTION tdmclient does not support yet multiple and/or in if statements!!!! 
        if prox_front > OBSTACLE_THRESH_HIGH:  
            print("OBSTACLE")
            if prox_left + prox_left_front < prox_right + prox_right_front:
                node.send_set_variables(motors(-300, 300))  
            else:
                node.send_set_variables(motors(300, -300))  
        elif prox_left_front > OBSTACLE_THRESH_HIGH:  
            node.send_set_variables(motors(200, -200))  
        elif prox_right_front > OBSTACLE_THRESH_HIGH:  
            node.send_set_variables(motors(-200, 200))  
        elif prox_left > OBSTACLE_THRESH_HIGH:  
            node.send_set_variables(motors(300, 100))  
        elif prox_right > OBSTACLE_THRESH_HIGH: 
            node.send_set_variables(motors(100, 300)) 
        elif prox_right_front > OBSTACLE_THRESH_HIGH: 
            node.send_set_variables(motors(0,0))
        elif prox_right_front > OBSTACLE_THRESH_HIGH: 
            node.send_set_variables(motors(0,0))
        elif prox_front > OBSTACLE_THRESH_LOW: 
            node.send_set_variables(motors(100, 100)) 
        #else:
        #    node.send_set_variables(motors(300, 300))  
    except KeyError:
        print("No update sent.")


1. What Is node.client.loop.create_task?
node.client:

This refers to the ClientAsync object provided by tdmclient. It is the central interface for managing communication with the Thymio.
loop:

loop refers to the event loop provided by Python's asyncio library. This is where all asynchronous tasks (e.g., motor control, variable updates) are scheduled and run.
create_task:

This is a method of the asyncio event loop that schedules a coroutine (an async def function) to run concurrently.
It allows you to start an asynchronous function from within a synchronous context, such as a callback.
How It Fits Together
When you call node.client.loop.create_task, you are:

Accessing the event loop running in the ClientAsync instance.
Scheduling an async def function (a coroutine) to run in the background.

2. Why Use create_task?
In your case, you need to use create_task because:

The callback on_variables_changed is synchronous (non-async).
You want to call an async def function (handle_variable_change_async) to use await.
create_task allows you to bridge this gap by scheduling the asynchronous function to run without blocking the synchronous callback.



In [110]:
node.set_variables(motors(0, 0))

<generator object ClientAsyncNode.set_variables at 0x1146db140>

In [113]:
async def main():
    client = ClientAsync()  # Initialize the client
    node = await client.wait_for_node()  # Wait for the robot to connect
    await node.lock()  # Lock the node for safe access

    try:
        while (True):
            await node.watch(variables=True)  # Watch all variables
            node.add_variables_changed_listener(on_variables_changed)
            await circular_motion(node) #gets back to turning on himself 
            #print("Program is running. Stop the kernel to stop the program.")
            await client.sleep()  # Keep the program running
    except Exception as e:
        print(f"Error: {e}")
    finally:
        print("Stopping motors...")
        await stop_motors(node)  # Stop the motors before exiting
        await node.unlock()  # Unlock the node
        await client.close()  # Close the client connection


In [115]:
# Run the main function
await main()


circular motion
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No update sent.
No updat

TypeError: object NoneType can't be used in 'await' expression

In [86]:
node.unlock()

<generator object ClientAsyncNode.unlock at 0x1146d8e40>

In [46]:
#couldn't stop the robot with this config when stopping kernel 
""" with ClientAsync() as client: 
    async def prog():
        with await client.lock() as node:
            await node.watch(variables=True)
            node.add_variables_changed_listener(on_variables_changed)
            await client.sleep() #keeps the program running 
    client.run_async_program(prog) """


prox.horizontal not found in variables. No update sent.
Obstacle detected! Turning right.
prox.horizontal not found in variables. No update sent.
prox.horizontal not found in variables. No update sent.
Obstacle detected! Turning right.
prox.horizontal not found in variables. No update sent.
prox.horizontal not found in variables. No update sent.
Path clear. Moving forward.
prox.horizontal not found in variables. No update sent.
prox.horizontal not found in variables. No update sent.
Path clear. Moving forward.
prox.horizontal not found in variables. No update sent.
prox.horizontal not found in variables. No update sent.
Path clear. Moving forward.
prox.horizontal not found in variables. No update sent.
prox.horizontal not found in variables. No update sent.
Path clear. Moving forward.
prox.horizontal not found in variables. No update sent.
prox.horizontal not found in variables. No update sent.
Path clear. Moving forward.
prox.horizontal not found in variables. No update sent.
prox.hor

KeyboardInterrupt: 

In [98]:

await main()


Starting straight trajectory...
Following straight trajectory...


TypeError: object NoneType can't be used in 'await' expression

## Potential fields: 

In [ ]:
""" SPEED0 = 100  # Nominal speed
OBST_SPEED_GAIN = [6, 4, -2, -6, -8]  # Gains for obstacle avoidance

def potential_field_control(node , variables):
    """
    #Asynchronous potential field algorithm for obstacle avoidance and goal tracking.
    """
    while True:
        try:
            prox = variables["prox.horizontal"]
            prox_front = prox[2]
            prox_left = prox[0]
            prox_left_front = prox[1]
            prox_right_front = prox[3]
            prox_right = prox[4]
            prox_behind_right = prox[5]
            prox_behind_left = prox[6]
            
            
            # Calculate the ground gradient adjustment
            diffDelta = prox_ground_delta[1] - prox_ground_delta[0]
            spLeft = SPEED0 - SPEED_GAIN * diffDelta
            spRight = SPEED0 + SPEED_GAIN * diffDelta
            
            # Adjust speeds based on proximity sensors
            for i in range(5):
                spLeft += prox[i] * OBST_SPEED_GAIN[i] // 100
                spRight += prox[i] * OBST_SPEED_GAIN[4 - i] // 100
            
            # Apply the calculated motor speeds
            node.send_set_variables(motors(spLeft, spRight))
            
        except Exception as e:
            print(f"Error in potential field control: {e}")
        
        # Sleep to avoid flooding with updates
        #await ClientAsync.sleep(0.01)  # 10ms sampling time


async def main():
    client = ClientAsync()  # Initialize the client
    node = await client.wait_for_node()  # Wait for the robot to connect
    await node.lock()  # Lock the node for safe access

    try:
        while (True):
            await node.watch(variables=True)  # Watch all variables
            node.add_variables_changed_listener(on_variables_changed)
            await circular_motion(node) #gets back to turning on himself 
            #print("Program is running. Stop the kernel to stop the program.")
            await client.sleep()  # Keep the program running
    except Exception as e:
        print(f"Error: {e}")
    finally:
        print("Stopping motors...")
        await stop_motors(node)  # Stop the motors before exiting
        await node.unlock()  # Unlock the node
        await client.close()  # Close the client connection
 """

In [ ]:
await main()

In [ ]:
#Potential field with help of chatgpt


# Constants
SPEED0 = 300  # Nominal speed
SPEED_GAIN_ATTRACTIVE = 2  # Gain for attractive force
OBST_SPEED_GAIN = [6, 4, -2, -6, -8]  # Gains for obstacle avoidances
GOAL_X, GOAL_Y = 100, 100  # 2D coordinates of the goal , units? 
TIME_STEP = 0.1  # Time step for updates 
WAYPOINT_TOLERANCE = 5  # Tolerance to determine if a waypoint is reached? 


def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

def calculate_attractive_force(robot_x, robot_y, goal_x, goal_y):
    """
    Calculate the attractive force vector toward the goal.
    """
    dx = goal_x - robot_x
    dy = goal_y - robot_y
    distance = math.sqrt(dx**2 + dy**2) + 1e-6  # Avoid division by zero
    force_x = dx / distance  
    force_y = dy / distance  
    return force_x, force_y

async def potential_field_control(node, waypoint):
    """
    Implements potential field-based local navigation toward a given waypoint.
    """
    robot_x, robot_y = 0, 0  # Initial robot position (arbitrary units)
    robot_heading = 0  # Robot's initial heading

    while True:
        try:
            # Ensure variables are updated
            await node.wait_for_variables({"prox.horizontal"})

            # Proximity sensor readings
            prox = node.v.prox.horizontal
            prox_front = prox[2]
            prox_left = prox[0]
            prox_left_front = prox[1]
            prox_right_front = prox[3]
            prox_right = prox[4]

            # Check if the robot has reached the waypoint
            dx = waypoint[0] - robot_x
            dy = waypoint[1] - robot_y
            distance_to_waypoint = math.sqrt(dx**2 + dy**2)

            if distance_to_waypoint < WAYPOINT_TOLERANCE:
                print(f"Waypoint {waypoint} reached!")
                return True  # Notify the global planner

            # Calculate attractive force toward the waypoint
            attractive_force_x, attractive_force_y = calculate_attractive_force(
                robot_x, robot_y, waypoint[0], waypoint[1]
            )

            # Convert attractive force to motor speeds
            spLeft = SPEED0 + SPEED_GAIN_ATTRACTIVE * attractive_force_y
            spRight = SPEED0 - SPEED_GAIN_ATTRACTIVE * attractive_force_y

            # Add obstacle avoidance adjustments
            for i in range(5):
                spLeft += prox[i] * OBST_SPEED_GAIN[i] // 100
                spRight += prox[i] * OBST_SPEED_GAIN[4 - i] // 100

            # Apply the calculated motor speeds
            await node.set_variables(motors(int(spLeft), int(spRight)))

            # Update robot's position (simple odometry simulation)
            robot_heading += (spRight - spLeft) / 1000  # Adjust heading
            robot_x += SPEED0 * math.cos(robot_heading) * TIME_STEP
            robot_y += SPEED0 * math.sin(robot_heading) * TIME_STEP

            print(f"Robot position: ({robot_x:.2f}, {robot_y:.2f}), Heading: {robot_heading:.2f}")

        except Exception as e:
            print(f"Error in potential field control: {e}")

        # Wait for the next update
        await asyncio.sleep(TIME_STEP)
        
async def main():
    client = ClientAsync()
    node = await client.wait_for_node()
    await node.lock()

    # Example global planner output (sequence of waypoints)
    waypoints = [(50, 50), (100, 100), (150, 100)]

    try:
        print("Starting navigation...")
        for waypoint in waypoints:
            print(f"Navigating to waypoint {waypoint}...")
            reached = await potential_field_control(node, waypoint)
            if not reached:
                print(f"Failed to reach waypoint {waypoint}.")
                break
    except KeyboardInterrupt:
        print("Program interrupted. Stopping...")
    finally:
        print("Stopping motors...")
        await node.set_variables(motors(0, 0))  # Stop motors
        await node.unlock()
        await client.close()


In [ ]:

await main()
